<h1 align="center"><b> Lie point symmetries for first order ODEs </b></h1>
 <h3 align="center">A symbolic algorithm for the maxima CAS </h3>


<h3 align="left"><b>The method of undetermined coefficients </b></h3>


The method of undetermined coefficients (symmetry5) is the most successful methods of the paper of Cheb-terrab and Roche [2]. In the paper, the symmetry generators $\xi$ and $\eta$ are assumed to be bivariate: $\xi=a_1x+a_2y+a_3$, $\eta=b_1x+b_2y+b_3$ and the coefficients $a_1..a_3,b_1..b_3$ are solved using the method of undetermined coefficients. The default in our maxima implementation is a bivariate polynomial of degree 2. For $\xi$ this means $\xi=a_1x + a_2y + a_3 + a_4x^2 + a_5xy + a_6y^2$. This means we can solve many more odes with the method of undetermined coefficients than mentioned in the paper. 
we load the file containing the list of Kamke ODEs and the file ode1_lie.mac

In [ ]:
kill(all);batch("~/mathematics/maxima_files/kamke1_1.mac");batch("~/mathematics/maxima_files/ode1_lie.mac");

In [2]:
ode:kamke1[23];

                                 dy      2
(%o156)                          -- + a y  - b
                                 dx

In [7]:
DEBUGFLAG:1;returnSymmetries:true;returnIntegratingFactor:true;returnSolution:true;
assume(a*b>0);
ode1solve(ode,y,x,'useMethod="muc");
forget(a*b>0);
method;

(%o184)                                1

(%o185)                              true

(%o186)                              true

(%o187)                              true

(%o188)                           [redundant]

                                        b        2 sqrt(a b) x
                                   sqrt(-) (%c %e              + 1)
                      1                 a
(%o189)  [[1, 0], [--------], [y = --------------------------------]]
                      2                      2 sqrt(a b) x
                   a y  - b             %c %e              - 1

(%o190)                            [a b > 0]

(%o191)                        [ax+by+c,fx+gy+h]

The ODE has a simple polynomial symmetry due to being separable, but also a polynomial symmetry of degree 2, which is given in the first item of the output list. 

Polynomial symmetries occur frequently and the method of undetermined coefficients is one of the most successfull algorithms for finding symmetries. 
The method of undetermined coefficients sees rational variables like log(), sqrt() and exp() as additional variables. A multivariate polynomial is constructed that includes these variables. For instance for Kamke ODE 1.59: 

In [8]:
ode:kamke1[59];

                            dy           2
(%o192)                     -- - a sqrt(y  + 1) - b
                            dx

This ODE has a square root term. First, we try to factor the term. In this case, we can factor into $\sqrt{y^2+1}=\sqrt{y+i}\sqrt{y-i}$. The factors are complex, and we value real over complex. This is why we ignore the complex factorization and introduce the new variable $z=\sqrt{y^2+1}$ and we construct a second degree polynomial in $x,y,z$. Note that if the ode had a term of the form $\sqrt{y^2-1}=\sqrt{y+1}\sqrt{y-1}$, we would have introduced 2 new variables $z_1=\sqrt{y+1}$ and $z_2=\sqrt{y-1}$. Factorization gives us more variables and a more general shape of the determining equations.

In [9]:
ode1solve(ode,y,x,'useMethod="symmetry5");

inline-value

                     2
             a sqrt(y  + 1) + b            1
(%o193) [[1, ------------------], [------------------], 
                     a                     2
                                   a sqrt(y  + 1) + b
                                             /
                                             [         1
                                            [I ------------------ dy - x = %c]]
                                             ]         2
                                             / a sqrt(y  + 1) + b

The method of undetermined coefficients finds a polynomial symmetry and an implicit solution (we asked for an explicit solution, hence the warning).


Additionally, the method of undetermined coefficients searches for rational polynomial symmetries of the form $\xi=Q_1/P_1$ and $\eta=Q_2/P_2$, with $P,Q$ bivariate polynomials. Because the search for such symmetries is much more expensive, the default is to search only for rational symmetries of the form $[\xi,\eta]=[0,\frac{Q}{P}]$ and $[\xi,\eta]=[\frac{Q}{P},0]$. It turns out that there is a class of ODEs of the form $y'=\frac{Q(x,y)}{P(x,y)}$ and $y'=\frac{Q(x,y)}{xP(x,y)}$ whose symmetries have the form $[\xi,\eta]=[0,\frac{F(x,y)}{P(x,y)}]$, so symmetries whose denominator is the denominator (or denominator times $x$) of the right-hand side of the differential equation. We are also looking for these symmetries. 

In [10]:
ode:kamke1[346];

             dy
(%o194)    x -- (y log(x y) + y - a x) - y (a x log(x y) - y + a x)
             dx

This ODE can be written as $y'=Q/P$, with $P=x(\log(xy)+y-ax)$. So we are factoring out the $x$, and are looking for symmetries of the form $[\xi,\eta] =[0,\frac{F(x,y)}{\log(xy)+y-ax}]$. The method *symmetry5b* will look for such symmetries.

In [11]:
ode1solve(ode,y,x,'useMethod="symmetry5b");

inline-value

                       y                1
(%o195) [[0, ----------------------], [---], [(y - a x) log(x y) = %c]]
             y (log(x y) + 1) - a x    x y

note that the denominator of the symmetry does not match the original denominator of the ODE. This is because internally, ode1solve tries to simplify the ODE by rewriting $\log(xy)$ in terms of $\log(x)$ and $\log(y)$. These canonical forms can be obtained as follows:

In [12]:
ode1CanonicalForm(ode,y,x);

                        dy   y (a x log(x y) - y + a x)
(%o196)                [-- = --------------------------]
                        dx    x (y log(x y) + y - a x)

In [13]:
P:factor_list(denom(rhs(%[1])));

(%o197)                    [x, y log(x y) + y - a x]

# user defined functions for symmetries

symmetry5 can be used to look for user defined shapes of symmetries. The user has to supply a function with unknown coefficients for the symmetries $\xi,\eta$. Let's take a look at Kamke ODE 1.62, which cannot be solved automatically by ode1solve:

In [14]:
ode:kamke1[62];

                                    2       2    2
                          dy   y - x  sqrt(x  - y )
(%o198)                   -- = ---------------------
                          dx             2    2
                               x y sqrt(x  - y ) + x

In [15]:
returnSolution:false;returnIntegratingFactor:false;returnSymmetries:true;DEBUGFLAG:1;

(%o199)                              false

(%o200)                              false

(%o201)                              true

(%o202)                                1

In [16]:
ode1solve(ode,y,x);


expt: undefined: 0 to a negative exponent.

expt: undefined: 0 to a negative exponent.

inline-value

(%o203)                              false

We did not find any nontrivial symmetries of the ODE. Actually, we found some trivial symmetries, which internally lead to the *expt: undefined: 0 to a negative exponent* error message. We can use the method of undetermined coefficients with a user defined function for the shape of the symmetries. The ODE contains the term $\sqrt{x^2-y^2}$. If the symmetry contains such a term, it will not be recognized by any of the methods. With the method of undetermined coefficients, we can look for a symmetry containing this term. Let's define a shortcut:

In [17]:
V1:sqrt(x^2-y^2);

                                       2    2
(%o204)                          sqrt(x  - y )

Let's also define a shortcut for $\sqrt{x^2+y^2}$:

In [18]:
V2:sqrt(x^2+y^2);

                                       2    2
(%o205)                          sqrt(y  + x )

We could define a very general multivariate polynomial with the variables $x,y,V_1,V_2$, but after some trial and error you will find that this one works:

In [19]:
X:ode1solve(ode,y,x,'useMethod="symmetry5",'xi=a1*x/V1^2+a2*x/V2^2,'eta=b1*y/V1^2+b2*y/V2^2);

                                  x        y
(%o206)                       [-------, -------]
                                2    2   2    2
                               y  + x   y  + x

In [20]:
print("X=",X,"is a symmetry of ODE ",ode,": ", is(checkSymmetries(X,ode,y,x)=0))$

inline-value

we could have tried a symmetry search for a general rational symmetry of degree 2. However, this computation is very expensive. The search for such symmetries is not done automatically. With the symmetry that we have just found, we can construct an integrating factor for the ODE. Note that our symmetry is valid for the canonical form of the ODE, but in this case the ODE is already in canonical form. If it is not, you can use the function *ode1CanonicalForm(ode,y,x)* to place the ODE in canonical form.

In [21]:
ode:ode1CanonicalForm(ode,y,x)[1];
mu:ode1IntegratingFactor(X,ode,y,x);
isIntegratingFactor(mu,ode,y,x);
mu:ratsimp(%i*mu);
isIntegratingFactor(mu,ode,y,x);

                                 2       2    2
                        dy      x  sqrt(x  - y ) - y
(%o208)                 -- = - -----------------------
                        dx                2    2
                               x (y sqrt(x  - y ) + 1)

                                      %i
(%o209)                         ---------------
                                        2    2
                                x sqrt(y  - x )

(%o210)                              true

                                        1
(%o211)                        - ---------------
                                         2    2
                                 x sqrt(y  - x )

(%o212)                              true

A manual computation of the integrating factor from the symmetry will reveal that the integrating factor is complex. It turns out that if we multiply the integrating factor with $i$, it is still an integrating factor, which might lead to a more elegant solution later in the computation. Internally, we try this if the integrating factor is complex. Unfortunately, computing an elegant solution using this integrating factor was not possible, but a solution can certainly be computed using *ode1SolveWithIntegratingFactor(ode,mu,y,x,returnExplicit)* where (bool) returnExplicit=true means that we want to compute an explicit solution.

# Bibliography
[1] E.S. Cheb-Terrab and T. Kolokolnikov, First-order ordinary differential equations, symmetries and linear transformations, Euro. J. of Applied Mathematics 14 (2003)

[2] E.S. Cheb-Terrab and A.D. Roche, Symmetries and first order ODE patterns, Computer Physics Communications 113 (1998)

[3] E.S. Cheb-Terrab, L.G.S. Duarte and L.A.C.P. da Mota, Computer algebra solving of first order ODEs using symmetry methods, Computer Physics Communications 101 (1997)

[4] F. Schwarz, Symmetry analysis of Abel's equation, Studies in applied mathematics 100 (1998)

[5] F. Schwarz, Algorithmic solution of Abel's equation, Computing 61 (1998)

[6] E. Kamke, Differentialgleichungen, L$\mathrm{\ddot o}$sungsmethoden und L$\mathrm{\ddot o}$sungen, Leipzig (1959)
